## Load our models

In [ ]:
import cv2
import numpy as np
import pandas as pd

# to create our interface
import gradio as gr

# to load our models
from keras.models import load_model
import joblib

## Class Representation

In [ ]:
# Reading the file
label_map = pd.read_csv("./dataset/EMNIST/emnist-balanced-mapping.txt",
                        delimiter=' ',
                        index_col=0,
                        header=None)
label_map = label_map.iloc[:, 0]

# Initialising an empty dictionary
label_dictionary = {}

# Running a loop for ASCII equivalent to character conversion
for index, label in enumerate(label_map):
    label_dictionary[index] = chr(label)

## Load our models

In [ ]:
#Deep Learning model
cnn_model = load_model('./saved/cnn.h5')

# Machine Learning models
svm_model = joblib.load('./saved/svm.pkl')
rf_model = joblib.load('./saved/random_forest.pkl')
knn_model = joblib.load('./saved/knn.pkl')
lg_model = joblib.load('./saved/logistic_regression.pkl')
nb_model = joblib.load('./saved/naive_bayes.pkl')

models = [
    cnn_model,
    svm_model,
    rf_model,
    knn_model,
    lg_model,
    nb_model
    ]

## Creating models name for dropdown-list model

In [ ]:
model_names = [
    "CNN",
    "SVM",
    "Random forest",
    "KNN",
    "Logistic regression",
    "Naive-Bayes"
    ]

## Preprocess image or sketchpad drawing

In [ ]:
def process_image(image, model_name):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    average_color = np.mean(gray)
    if average_color > 127:
        inverted = cv2.bitwise_not(gray)
        processed_image = inverted
    else:
        processed_image = gray

    resized = cv2.resize(processed_image, (28, 28), interpolation=cv2.INTER_CUBIC)
    
    if model_name == "CNN":
        image_array = resized.reshape((1, 28, 28, 1))
        image_array = image_array.astype('float32') / 255
    else:
        image_array = resized.reshape((1, 784))
        image_array = image_array.astype('float32')

    return image_array, resized

## Get predictions

In [ ]:
def prediction(sketchInput, model_name):
    model_index = model_names.index(model_name)
    
    # Convert sketch to image
    sketch = sketchInput.copy()
    
    sketch = cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR) # this is specific to sketchpad ui, delete it when working on image ui

    # Preprocess image
    testImage, resized = process_image(sketch, model_name)
    loaded_model = models[model_index]
    if model_index == 0:
        prediction = np.argmax(loaded_model.predict(testImage))
        output = label_dictionary[prediction]
    else:
        prediction = loaded_model.predict(testImage)
        output = label_dictionary[prediction[0]]

    return f'{output}'
    

## User Interface: Creation

In [ ]:
# This interface allow te user to draw a character in the sketchapd, choose a model and get the prediction
sketchpad_ui = gr.Interface(
    fn=prediction, inputs=["sketchpad", gr.inputs.Dropdown(model_names)], 
    outputs="label"
)
sketchpad_ui.launch()

In [ ]:
# This allows the user to upload an image containing a character to predict after choosing a model from dropdown-list
image_ui = gr.Interface(
    fn=prediction, inputs=["image", gr.inputs.Dropdown(model_names)], 
    outputs="label"
)
image_ui.launch()